In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import io
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
df = pd.read_csv("/content/drive/MyDrive/롯데 공모전/dataset/lotto_df.csv") 

In [4]:
df = df.set_index(['cust', 'br_c'])
df.head()

Unnamed: 0    index        rct_no  chnl_dv cop_c    pd_c  \
cust       br_c                                                                
M000034966 A043676     3094539   728698  A04003723283        1   A04  PD1161   
           A043676     3094540  2024520  A04003723283        1   A04  PD1330   
           A020116     3094565  2749972  A02357415245        1   A02  PD0116   
           A044861     3094547  1856107  A04081271994        1   A04  PD1165   
           A020116     3094536   786013  A02358961675        1   A02  PD0218   

                         de_dt  de_hr   buy_am  buy_ct  ... day month  \
cust       br_c                                         ...             
M000034966 A043676  2021-01-16     21   1200.0       1  ...  16     1   
           A043676  2021-01-16     21  10800.0       4  ...  16     1   
           A020116  2021-03-28     21   6960.0       2  ...  28     3   
           A044861  2021-08-29     15   1400.0       1  ...  29     8   
           A020116  2021-09-19     16   3980.0       1  ...  19     9   

                   total_items one_times_items rct_cnt rebuy  rebuy_cnt  \
cust       br_c                                                           
M000034966 A043676          45               2       5     0         19   
           A043676          45               2       5     1         19   
           A020116          45               1       5     1         19   
           A044861          45               1       5     1         19   
           A020116          45              24       5     1         19   

                    rebuy_ration  total_am  dt_diff  
cust       br_c                                      
M000034966 A043676     42.222222    1200.0      NaN  
           A043676     42.222222   43200.0      0.0  
           A020116     42.222222   13920.0     71.0  
           A044861     42.222222    1400.0    154.0  
           A020116     42.222222    3980.0     21.0  

[5 rows x 26 columns]

In [5]:
df =  df.drop(['Unnamed: 0', 'index', 'rct_no','de_dt','pd_nm','clac_hlv_nm', 'clac_mcls_nm','pd_c','cop_c'], axis=1)
df.head()

chnl_dv  de_hr   buy_am  buy_ct ma_fem_dv ages zon_hlv  \
cust       br_c                                                              
M000034966 A043676        1     21   1200.0       1        여성  40대     Z07   
           A043676        1     21  10800.0       4        여성  40대     Z07   
           A020116        1     21   6960.0       2        여성  40대     Z07   
           A044861        1     15   1400.0       1        여성  40대     Z07   
           A020116        1     16   3980.0       1        여성  40대     Z07   

                    day  month  total_items  one_times_items  rct_cnt  rebuy  \
cust       br_c                                                                
M000034966 A043676   16      1           45                2        5      0   
           A043676   16      1           45                2        5      1   
           A020116   28      3           45                1        5      1   
           A044861   29      8           45                1        5      1   
           A020116   19      9           45               24        5      1   

                    rebuy_cnt  rebuy_ration  total_am  dt_diff  
cust       br_c                                                 
M000034966 A043676         19     42.222222    1200.0      NaN  
           A043676         19     42.222222   43200.0      0.0  
           A020116         19     42.222222   13920.0     71.0  
           A044861         19     42.222222    1400.0    154.0  
           A020116         19     42.222222    3980.0     21.0

In [6]:
ages_list = df['ages'].value_counts().index[::-1]
ages_dict = dict(zip(ages_list, range(0,17)))
df = df.replace({'ages': ages_dict})

In [7]:
ma_fem_list = df['ma_fem_dv'].value_counts().index[::-1]
ma_fem_dict = dict(zip(ma_fem_list, range(0,2)))
df = df.replace({'ma_fem_dv': ma_fem_dict})

In [8]:
zon_hlv_list =df['zon_hlv'].value_counts().index[::-1]
zon_hlv_dict = dict(zip(zon_hlv_list, range(0,17)))
df = df.replace({'zon_hlv': zon_hlv_dict})

In [9]:
df = df.fillna(0)

In [10]:
X = df.drop(['rebuy'], axis=1)
y = df['rebuy']

In [11]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [12]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.3,random_state=2)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(2209832, 16)
(2209832,)
(947071, 16)
(947071,)


In [13]:
np.unique(y,return_counts=True)

(array([0, 1]), array([ 526055, 2630848]))

In [14]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

### RandomForestRegressor

In [15]:
forest = RandomForestClassifier(criterion='gini', n_estimators=25, random_state=1)

forest.fit(X_train, y_train)
y_train_pred = forest.predict(X_train)
y_test_pred = forest.predict(X_test)

print('훈련 MSE: %.3f, 테스트 MSE: %.3f' % (
  mean_squared_error(y_train, y_train_pred),
  mean_squared_error(y_test, y_test_pred)))
print('훈련 R^2: %.3f, 테스트 R^2: %.3f' % (
      r2_score(y_train, y_train_pred),
      r2_score(y_test, y_test_pred)))
print('훈련 정확도 정확도: %.3f, 테스트 정확도: %.3f' % (
      accuracy_score(y_train, y_train_pred),
      accuracy_score(y_test, y_test_pred)))

훈련 MSE: 0.009, 테스트 MSE: 0.164
훈련 R^2: 0.938, 테스트 R^2: -0.182
훈련 정확도 정확도: 0.991, 테스트 정확도: 0.836


In [16]:
forest = RandomForestClassifier(criterion='gini', n_estimators=50, random_state=1)

forest.fit(X_train, y_train)
y_train_pred = forest.predict(X_train)
y_test_pred = forest.predict(X_test)

print('훈련 MSE: %.3f, 테스트 MSE: %.3f' % (
  mean_squared_error(y_train, y_train_pred),
  mean_squared_error(y_test, y_test_pred)))
print('훈련 R^2: %.3f, 테스트 R^2: %.3f' % (
      r2_score(y_train, y_train_pred),
      r2_score(y_test, y_test_pred)))
print('훈련 정확도 정확도: %.3f, 테스트 정확도: %.3f' % (
      accuracy_score(y_train, y_train_pred),
      accuracy_score(y_test, y_test_pred)))

훈련 MSE: 0.007, 테스트 MSE: 0.162
훈련 R^2: 0.951, 테스트 R^2: -0.169
훈련 정확도 정확도: 0.993, 테스트 정확도: 0.838


In [17]:
forest = RandomForestClassifier(criterion='gini', n_estimators=75, random_state=1)

forest.fit(X_train, y_train)
y_train_pred = forest.predict(X_train)
y_test_pred = forest.predict(X_test)

print('훈련 MSE: %.3f, 테스트 MSE: %.3f' % (
  mean_squared_error(y_train, y_train_pred),
  mean_squared_error(y_test, y_test_pred)))
print('훈련 R^2: %.3f, 테스트 R^2: %.3f' % (
      r2_score(y_train, y_train_pred),
      r2_score(y_test, y_test_pred)))
print('훈련 정확도 정확도: %.3f, 테스트 정확도: %.3f' % (
      accuracy_score(y_train, y_train_pred),
      accuracy_score(y_test, y_test_pred)))

훈련 MSE: 0.007, 테스트 MSE: 0.161
훈련 R^2: 0.952, 테스트 R^2: -0.163
훈련 정확도 정확도: 0.993, 테스트 정확도: 0.839


### LogisticRegression

In [24]:
LR = LogisticRegression(C=1e2, multi_class='ovr', random_state=1, max_iter=200)

LR.fit(X_train, y_train)
y_train_pred = LR.predict(X_train)
y_test_pred = LR.predict(X_test)

print('훈련 MSE: %.3f, 테스트 MSE: %.3f' % (
      mean_squared_error(y_train, y_train_pred),
      mean_squared_error(y_test, y_test_pred)))
print('훈련 R^2: %.3f, 테스트 R^2: %.3f' % (
      r2_score(y_train, y_train_pred),
      r2_score(y_test, y_test_pred)))

print('훈련 정확도: %.3f, 테스트 정확도: %.3f' % (
     accuracy_score(y_train, y_train_pred),
      accuracy_score(y_test, y_test_pred)))

훈련 MSE: 0.160, 테스트 MSE: 0.160
훈련 R^2: -0.151, 테스트 R^2: -0.153
훈련 정확도: 0.840, 테스트 정확도: 0.840


### 서포트 벡터

In [30]:
import sys
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
# make a classifier and fit on training data
clf = svm.SVC(kernel='linear')

# Train classifier 
clf.fit(X_train, y_train)

# Make predictions on unseen test data
clf_predictions = clf.predict(X_test)
print("Accuracy: {}%".format(clf.score(X_test, y_test) * 100 ))